# EDA : Brazilian E-Commerce Public Dataset

### 1. 데이터 전처리

In [13]:
### 데이터 조인을 통한 다수 데이터셋 통합 진행 
### 아래 SQL 쿼리문 참고, order_id는 중복 허용됨, order_status에서 cancel된 주문은 제외

# create or replace view order_info as
# select a.order_id,
# 	b.customer_id,
#   d.customer_unique_id,
# 	d.customer_city,
# 	d.customer_state,
# 	d.customer_zip_code_prefix as customer_zipcode,
# 	c.product_category_name_english as category,
# 	b.order_status,
# 	b.order_purchase_timestamp as purchase_date,
# 	b.order_estimated_delivery_date as estimated_delivery_date,
# 	datediff(b.order_estimated_delivery_date, b.order_purchase_timestamp) as estimated_delivery_period,
# 	a.price + a.freight_value as total_price
# from olist_order_items_dataset a
# left join olist_orders_dataset b on a.order_id = b.order_id
# left join product_dataset c on a.product_id = c.product_id
# left join olist_customers_dataset d on b.customer_id = d.customer_id
# having b.order_status != 'canceled';

In [2]:
import mysql.connector
import pandas as pd
import plotly_express as px
from datetime import datetime, timedelta

# mysql 데이터베이스 연결
conn = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "cx@1076044150",
    database = "olist"
)
cursor = conn.cursor(buffered=True)


# 쿼리한 데이터셋 불러오기
sql = 'select * from order_info'
cursor.execute(sql)
result = cursor.fetchall()
columns = [col[0] for col in cursor.description]
df = pd.DataFrame(data=result, columns=columns)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112108 entries, 0 to 112107
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   order_id                   112108 non-null  object        
 1   customer_id                112108 non-null  object        
 2   customer_unique_id         112108 non-null  object        
 3   customer_city              112108 non-null  object        
 4   customer_state             112108 non-null  object        
 5   customer_zipcode           112108 non-null  int64         
 6   category                   110497 non-null  object        
 7   order_status               112108 non-null  object        
 8   purchase_date              112108 non-null  datetime64[ns]
 9   estimated_delivery_date    112108 non-null  datetime64[ns]
 10  estimated_delivery_period  112108 non-null  int64         
 11  total_price                112108 non-null  float64 

### 2. 재구매 고객 및 상품 카테고리 분석
- 재구매 기준 : olist몰의 판매제품의 사용연수가 다양한 종합몰이므로, 최초 구매 시기로부터 1년을 유효기간으로 가정하여 재구매를 정의
- 고객이 최초 구매 후 동일고객이 1년이내 재구매한 주문건수는 총 2169건 (구매일이 1일을 경과하지 않으면 재구매로 보지 않는다.)

In [113]:
# 재구매 고객 현황 (order_id 중복 제외)
df_re = pd.pivot_table(data=df, index='order_id', values=['customer_unique_id', 'purchase_date'], aggfunc='first')
df_re.sort_values(by='purchase_date', ascending=True, inplace=True)

# 고객별 최초 구매일 그룹핑
result = df_re.groupby('customer_unique_id')['purchase_date'].min()

# 1년이내 재구매 여부 확인
df_re['repurchase'] = df_re.apply(lambda row: 
                                  ((row['purchase_date'] - result[row['customer_unique_id']]).days >= 1) and
                                  ((row['purchase_date'] - result[row['customer_unique_id']]).days <= 365), axis=1)

print('1년이내 재구매된 주문건수 : ', len(df_re[df_re['repurchase'] == True]), '건 (구매일이 1일이상 경과하지 않으면 재구매로 보지 않는다.)')

1년이내 재구매된 주문건수 :  2169 건 (구매일이 1일이상 경과하지 않으면 재구매로 보지 않는다.)


In [134]:
# 재구매 카테고리 현황 (재구매 대상 order_id만 필터링)
df_re2 = df[df['order_id'].isin(df_re[df_re['repurchase'] == True].index)]

# 재구매 카테고리 구매빈도
df_re2 = pd.pivot_table(data=df_re2, index='category', values='customer_id', aggfunc='count')
df_re2 = df_re2.sort_values(by='customer_id', ascending=False).rename(columns={'customer_id':'order_count'})
df_re2 = df_re2[df_re2['order_count'] > 10]

px.bar(df_re2, x=df_re2.index, y='order_count',
       labels={"category":"카테고리","order_count":"주문건수"}, 
       title='카테고리별 재구매 주문건수')